## **Setup**

In [1]:
from google.colab import drive
mount_path = '/content/gdrive/'
drive.mount(mount_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4AFQf78QqcKytqwkRF5VENC6-jURSknHR9pqIWDmvkQ01oN-yYUAwZ8
Mounted at /content/gdrive/


In [2]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

#hide
!pip install utils
from utils import *

     |████████████████████████████████| 727kB 8.4MB/s 
     |████████████████████████████████| 1.0MB 13.4MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 358kB 43.0MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 92kB 12.3MB/s 
     |████████████████████████████████| 40kB 7.1MB/s 
     |████████████████████████████████| 51kB 8.1MB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 2.6MB 61.1MB/s 


In [3]:
  from fastai.vision.all import *

In [4]:
import os
import shutil
import glob
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime
import pytz
from zipfile import ZipFile
from tempfile import TemporaryDirectory
from PIL import Image

from sklearn.metrics import f1_score, accuracy_score

# plot options
# plt.rcParams.update({'font.size': 11})
plt.style.use('fivethirtyeight')

In [5]:
# Set and test path to competition data files
competition_path = 'My Drive/AI For Good - AI Blitz 3/Snake/Data/'
model_path='My Drive/AI For Good - AI Blitz 3/Snake/Models/'
results_path='My Drive/AI For Good - AI Blitz 3/Snake/Results/'
filename = 'train.zip'
assert(os.path.exists(f'{mount_path}/{competition_path}/{filename}'))
print('Drive mounted correctly and data accessible')

Drive mounted correctly and data accessible


# **Functions**



### **Data loading**

In [6]:
# custom copytree because Colab doesn't have the latest version of shutil.copytree which now contains the dirs_exist_ok flag solving this issue
# copied verbatim from https://stackoverflow.com/a/12514470/5991868
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)

# Unzip and combine images
Combine the train and validation sets from the competition organizers, we're going to use our own!


In [7]:
# this can take a bit (maybe 30s)
# All_Data is the home directory for the images, from which training and validation splits will be taken
if not os.path.exists('/content/All_Data/'):
  os.mkdir('/content/All_Data/')

  for filename in ['test']:
    # create a temporary directory using TemporaryDirectory and context manager and unzip to there
    with TemporaryDirectory() as tmpdirname:
      with ZipFile(f'{mount_path}/{competition_path}/{filename}.zip', 'r') as zip_ref:
        zip_ref.extractall(tmpdirname)
      # copy the data to the All_Data dir
      copytree(f'{tmpdirname}/content/data/{filename}/','/content/All_Data/')

# Make preds

In [8]:
def nlprint(lst):
  for ii in lst: print(ii)

In [9]:
# model_files=glob.glob(f'{os.path.join(mount_path,model_path)}*.pkl')
# print(model_files)

model_files=[
              '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_0917_resnet34_cv0.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1038_resnet34_cv1.pkl',              
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1158_resnet34_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1321_xresnet34_cv0.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1443_xresnet34_cv1.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1605_xresnet34_cv2.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1734_resnet50_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1904_resnet50_cv1.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2035_resnet50_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2227_xresnet50_cv0.pkl',
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2352_xresnet50_cv1.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0116_xresnet50_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0244_densenet121_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0410_densenet121_cv1.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0535_densenet121_cv2.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0726_densenet201_cv0.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0917_densenet201_cv1.pkl', 
             '/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_1109_densenet201_cv2.pkl']

model_list_1=model_files[:5]
model_list_2=model_files[5:10]

model_files.reverse()
model_list_3=model_files[:5]
model_list_4=model_files[5:10]

print('model_list_1')
print(len(model_list_1))
nlprint(model_list_1)

print('model_list_2')
print(len(model_list_2))
nlprint(model_list_2)

print('model_list_3')
print(len(model_list_3))
nlprint(model_list_3)

print('model_list_4')
print(len(model_list_4))
nlprint(model_list_4)

model_list_1
5
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_0917_resnet34_cv0.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1038_resnet34_cv1.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1158_resnet34_cv2.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1321_xresnet34_cv0.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1443_xresnet34_cv1.pkl
model_list_2
5
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1605_xresnet34_cv2.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1734_resnet50_cv0.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_1904_resnet50_cv1.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2035_resnet50_cv2.pkl
/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200830_2227_xresnet50_cv0.pkl
model_list_3
5
/content/gdrive/My Drive/AI For

### Order the image list

In [10]:
sampledf=pd.read_csv(f'{mount_path}{competition_path}sample_submission.csv')

In [11]:
imglst=['/content/All_Data/'+ii+'.jpg' for ii in sampledf['id']]

In [13]:
# for model in model_list_4[2:]:
model ='/content/gdrive/My Drive/AI For Good - AI Blitz 3/Snake/Models/200831_0535_densenet121_cv2.pkl'

learn = load_learner(model)

model_name=os.path.basename(model[:-4])
print(model_name)

tst_dl=learn.dls.test_dl(imglst)

learn.epoch=0
preds = learn.tta(dl=tst_dl,n=2)[0]

np.save(f'{mount_path}{results_path}{model_name}_test.npy', preds.numpy())

200831_0535_densenet121_cv2
